Notebook is a supplement to main. Caluclates time-independent maps of:
* daily max rainfall (20-year mean of annual/seasonal maximum total 1day precip.)
* daily max sfcWind (20-year mean of annual/seasonal maximum of the daily maximums)

once we have these, tack them onto the existing nc files produced by Main

In [3]:
import os
import glob
import pandas as pd
import numpy as np
import xarray as xr
from xmip.preprocessing import rename_cmip6
import matplotlib
import matplotlib.pyplot as plt
from nc_processing import calc_spatial_mean
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


### options

# Model
model = 'UKESM1-0-LL' 

# SAI reference baseline (to be taken from joined historical-into-ssp245 runs)
baseline_start, baseline_end = '2013','2032' # years are INCLUSIVE. gives warming of 1.49 in UKESM1, closest 20-year period to 1.5

# SAI assessment period
SAI_assessment_period_start, SAI_assessment_period_end = '2050', '2069' # years are INCLUSIVE

# ARISE ensemble_members. We also only use these same members for the baseline
ens_mems = ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2', 'r4i1p1f2', 'r8i1p1f2']

# seasons
seasons = ['DJF', 'MAM', 'JJA', 'SON']

# make SSP245 ensemble mean pr ds
def get_ssp245_ds(variable, table='Amon'):
    ds_list = []
    for es in ens_mems:
        path = '/badc/cmip6/data/CMIP6/ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/{e}/{t}/{v}/*/latest/'.format(e=es, t=table,v=variable)
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        
        path_hist = glob.glob('/badc/cmip6/data/CMIP6/*/*/UKESM1-0-LL/historical/{e}/{t}/{v}/*/latest/'.format(
        t=table, v=variable, e=es))[0]
        ds_hist = rename_cmip6(xr.open_mfdataset(path_hist+'*.nc'))    
        ds = xr.concat([ds_hist, ds], dim='time')
        ds = ds.sel(time=slice('1990', '2150'))
        if 'height' in ds.variables:
            ds = ds.drop('height')
        if 'type' in ds.variables:
            ds = ds.drop('type')
        ds_list.append(ds)
    
    DS = xr.concat(ds_list, dim='Ensemble_member')
    return DS

## similar, for PiControl
def get_pi(model, variable='tas', table='Amon'):
    dir_pi = glob.glob('/badc/cmip6/data/CMIP6/*/*/{m}/piControl/r1i*/{t}/{v}/*/latest/'.format(m=model, t=table, v=variable))
    files_pi = os.listdir(dir_pi[0])[0:3] # don't need the full length run
    paths_pi = []
    for x in files_pi:
        paths_pi.append(dir_pi[0]+x)
    ds = rename_cmip6(xr.open_mfdataset(paths_pi))
    if 'height' in ds.variables:
        ds = ds.drop('height')
    if 'type' in ds.variables:
        ds = ds.drop('type')
    return ds

## for ARISE
def get_ARISE_UKESM(variable='tas', table='Amon'):
    ds_list = []
    paths = glob.glob('/badc/deposited2022/arise/data/ARISE/MOHC/UKESM1-0-LL/arise-sai-1p5/*/{t}/{v}/*/*/'.format(
    t=table, v=variable))
    for path in paths:
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        if 'height' in ds.variables:
            ds = ds.drop('height')
        if 'type' in ds.variables:
            ds = ds.drop('type')
        ds_list.append(ds)
    DS = xr.concat(ds_list, dim='Ensemble_member')
    return DS


In [4]:
################
var = 'pr'
################

In [5]:
ds = get_ssp245_ds(variable=var, table='day')

In [6]:
ds

<xarray.Dataset>
Dimensions:      (time: 39960, Ensemble_member: 5, bnds: 2, y: 144, x: 192)
Coordinates:
  * time         (time) object 1990-01-01 12:00:00 ... 2100-12-30 12:00:00
  * y            (y) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * x            (x) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
Dimensions without coordinates: Ensemble_member, bnds
Data variables:
    time_bounds  (Ensemble_member, time, bnds) object dask.array<chunksize=(1, 9000, 2), meta=np.ndarray>
    lat_bounds   (Ensemble_member, time, y, bnds) float64 dask.array<chunksize=(1, 9000, 144, 2), meta=np.ndarray>
    lon_bounds   (Ensemble_member, time, x, bnds) float64 dask.array<chunksize=(1, 9000, 192, 2), meta=np.ndarray>
    pr           (Ensemble_member, time, y, x) float32 dask.array<chunksize=(1, 9000, 144, 192), meta=np.ndarray>
Attributes: (12/46)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  144000.0
    creation_date:          2019-06-24T14:59:51Z
    ...                     ...
    title:                  UKESM1-0-LL output prepared for CMIP6
    variable_id:            pr
    variant_label:          r1i1p1f2
    license:                CMIP6 model data produced by the Met Office Hadle...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d4dc0ebc-76bf-4013-947c-11f916d32972

In [8]:
ds_seasonal = ds.resample(time="QS-DEC").max()
out_baseline, out_baseline_seasonal = ds.sel(time=slice(baseline_start, baseline_end)), ds_seasonal.sel(time=slice(baseline_start, baseline_end))
out_baseline.attrs['t_bnds'] = [baseline_start, baseline_end]
out_baseline_seasonal.attrs['t_bnds'] = [baseline_start, baseline_end]

out_ssp245, out_ssp245_seasonal = ds.sel(time=slice(SAI_assessment_period_start, SAI_assessment_period_end)), ds_seasonal.sel(time=slice(SAI_assessment_period_start, SAI_assessment_period_end))
out_ssp245.attrs['t_bnds'] = [SAI_assessment_period_start, SAI_assessment_period_end]
out_ssp245_seasonal.attrs['t_bnds'] = [SAI_assessment_period_start, SAI_assessment_period_end]

In [ ]:
out_baseline, out_baseline_seasonal = xr.merge(ds_list), xr.merge(ds_list_seasonal, compat='override')